In [1]:
import warnings
from importlib import reload
from functools import partial, lru_cache

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
from joblib import delayed
from modAL import batch
from art.metrics import empirical_robustness
from art.attacks.evasion import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from ipynb.fs.defs import Bias
from ipynb.fs.defs.Datasets import generateData_twoPills_2D, generateData_twoPills_noNoise_2D, plot_dataset_2D

import libactive
import libadversarial
from libactive import MyActiveLearner, active_split
from libadversarial import adversarial, uncertainty, random_batch
from libutil import ProgressParallel
from libdatasets import *
import librun
from librun import run

In [4]:
X, _ = car()

In [19]:
matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        ("car", lambda: lru_cache()(car)())
    ],
    "dataset_mutators": {
        "none": (lambda *x: x),
    },
    "methods": [
         #bounds=[(np.amin(X[:,i]), np.amax(X[:,i])) for i in range(X.shape[1])]
        #("random", partial(random_batch, n_instances=10)),
        #("uncertainty", partial(batch.uncertainty_batch_sampling, n_instances=10)),
        ("deepfool", partial(adversarial, Attack=partial(DeepFool, verbose=False), n_instances=10)),
        ("deepfool logits", partial(adversarial, Attack=partial(DeepFool, verbose=False), n_instances=10, use_logits=True)),
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 0.1,
        "n_runs": 1
    }
}

In [ ]:
results = librun.run(matrix, force_run=True)

In [ ]:
librun.plot(results)

In [10]:
librun.table(results)

haberman
╒══════════════╤═══════════╤════════════╤═══════════════╤═══════════╕
│ method       │   AUC LAC │   AUC LF1C │   AUC AUC ROC │ max % @   │
╞══════════════╪═══════════╪════════════╪═══════════════╪═══════════╡
│ uncertainty  │    544.19 │     543.52 │        548.76 │ 167       │
├──────────────┼───────────┼────────────┼───────────────┼───────────┤
│ random       │    543.25 │     542.46 │        548.77 │ 237       │
├──────────────┼───────────┼────────────┼───────────────┼───────────┤
│ deepfool new │    543.22 │     542.4  │        548.84 │ -         │
├──────────────┼───────────┼────────────┼───────────────┼───────────┤
│ deepfool     │    543.21 │     542.39 │        548.84 │ -         │
╘══════════════╧═══════════╧════════════╧═══════════════╧═══════════╛
